In [ ]:
using Pkg; Pkg.activate(".")
using SteinMD
using Molly 
using PotentialLearning
using LinearAlgebra
using SpecialPolynomials

# plotting scripts - these are kept separate from the package for now
include("/Users/swyant/cesmix/dev/SteinMD.jl/src/makie/makie.jl")

In [ ]:
ref = MullerBrown()

In [ ]:
pce = PolynomialChaos(4, 2, ChebyshevU)

In [ ]:
# define properties
atom_mass = 1.0u"g/mol"
boundary = RectangularBoundary(Inf*u"nm")
temp = 100.0u"K"

# define initial system
atoms = [Atom(mass=atom_mass)]
coords = [SVector(-0.8, 1.2)u"nm"] # initial position
sys = System(
    atoms=atoms,
    coords=coords,
    boundary=boundary,
    general_inters=(ref,),
    loggers=(coords=CoordinateLogger(100; dims=2),),
)

# define simulator
sim_langevin = OverdampedLangevin(
            dt=0.002u"ps",
            temperature=temp,
            friction=4.0u"ps^-1")

In [ ]:
# run simulation - this will take a few seconds
simulate!(sys, sim_langevin, 5_000_000)

In [ ]:
# subsample to obtain training data
coords_train = [sys.loggers.coords.history[i][1] for i = 2:2000:length(sys.loggers.coords.history)]
ntrain = length(coords_train)
atoms_train = [Atom(mass=atom_mass) for i in 1:ntrain]

sys_train = [System(
    atoms=[atoms_i],
    coords=[coords_i],
    boundary=boundary,
    general_inters=(ref,),
    # k = 1.0u"kJ * K^-1 * mol^-1",
) for (atoms_i, coords_i) in zip(atoms_train, coords_train)]

In [ ]:
# ALT: subsample to obtain training data
x_train = [[-0.85, 1.23], [-0.92, 1.1], [-0.78, 1.46], [-0.82, 1.57], [-0.6, 1.27],
           [-0.51, 1.09], [-0.98, 1.72], [-0.54, 1.68], [-0.49, 1.26], [-0.80, 1.12]]
coords_train = [SVector(x[1], x[2])u"nm" for x in x_train]
ntrain = length(x_train)
atoms_train = [Atom(mass=atom_mass) for i in 1:ntrain]

# define initial ensemble
sys_train = [System(
    atoms=[atoms_i],
    coords=[coords_i],
    boundary=boundary,
    general_inters=(ref,),
    # k = 1.0u"kJ * K^-1 * mol^-1",
    loggers=(
        coords=CoordinateLogger(1; dims=2),
        ksd=StepComponentLogger(1; dims=2),
    )
) for (atoms_i, coords_i) in zip(atoms_train, coords_train)]

In [ ]:

# plot training samples
dist_units = u"nm"
xcoords = Vector(-3.5:0.05:1.5) .* dist_units
ycoords = Vector(-1.5:0.05:3.5) .* dist_units
ctr_grid = [xcoords, ycoords]
ctr_lvls = -150:20:400

f0 = plot_md_trajectory(sys, sys_train, ctr_grid, fill=true, lvls=ctr_lvls)

In [ ]:
train_potential!(sys_train,ref,pce)
pce.params

In [ ]:
ctr_lvls2 = -10000:200:2000
f0, ax0 = plot_contours_2D(pce,xcoords,ycoords; fill=true, lvls=ctr_lvls2)
coordmat = transpose(reduce(hcat, get_values(coords_train)))
scatter!(ax0, coordmat[:,1], coordmat[:,2], color=:red, label="training points")
axislegend(ax0)
f0

In [ ]:
rbf = RBF(Euclidean(2), β=1.0, ℓ = 0.1)
# define simulator
sim_svgd = StochasticSVGD(
            dt=0.001u"ps",
            kernel=rbf,
            temperature=temp, #1.0u"K",
            friction=4.0u"ps^-1")
     

In [ ]:
altrigger = TimeInterval(interval=200)

In [ ]:
# define initial ensemble
ens0 = [System(
    atoms=[atoms_i],
    coords=[coords_i],
    boundary=boundary,
    general_inters=(pce,),
    # k = 1.0u"kJ * K^-1 * mol^-1",
    loggers=(
        coords=CoordinateLogger(1; dims=2),
        ksd=StepComponentLogger(1; dims=2),
        # trigger=TriggerLogger(altrigger, 1),
        params=TrainingLogger(),
    )
) for (atoms_i, coords_i) in zip(atoms_train, coords_train)]

In [ ]:
ens = deepcopy(ens0) # I guess so that you constantly have access to the initialized ensemble
sys_final, alsteps = active_learn!(ens, sim_svgd, 2_000, sys_train, ref, altrigger)